# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [23]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [24]:
# !pip install gymnasium

In [25]:
# !pip install matplotlib

In [26]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [27]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


C:\Users\skfim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gymnasium\envs\registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [28]:
states = set(range(3))
actions = set(range(1))

print(f'{states=} | {actions=}')

T = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}
print(f'Transition function {T=}')

A_mdp = states, actions, T

states={0, 1, 2} | actions={0}
Transition function T={(0, 0): 1, (1, 0): 2, (2, 0): 2}


Попробуйте записать функцию перехода в матричном виде:

In [29]:
def dict_to_matrix(T):
    states = sorted({s for s, _ in T.keys()} | set(T.values()))
    n = len(states)
    P = np.zeros((n, n))

    for (s, a), s_next in T.items():
        P[s, s_next] = 1.0

    return P

P = dict_to_matrix(T)
print(P)

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [30]:
start = np.array([1, 0, 0], dtype=float)

steps = 5
Pn = np.linalg.matrix_power(P, steps)
dist = np.dot(start, Pn)

print(f"Распределение после {steps} шагов:", dist)
print("P(состояние=1):", dist[1])

# Приближение к стационарному распределению
limit_steps = 100
steady = start.dot(np.linalg.matrix_power(P, limit_steps))

print(f"\nПредел при N → ∞: {steady}")


Распределение после 5 шагов: [0. 0. 1.]
P(состояние=1): 0.0

Предел при N → ∞: [0. 0. 1.]


Задайте еще несколько MDP:

- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

In [31]:
#B:

states = set(range(4))
actions = set(range(3))

print(f'{states=} | {actions=}')

T_B = {
    (0, 0): 1,
    (0, 1): 2,
    (0, 2): 3,

    (1, 0): 1,
    (1, 1): 1,
    (1, 2): 1,

    (2, 0): 2,
    (2, 1): 2,
    (2, 2): 2,

    (3, 0): 3,
    (3, 1): 3,
    (3, 2): 3
}

print(f'Transition function {T_B=}')

B_mdp = states, actions, T_B


states={0, 1, 2, 3} | actions={0, 1, 2}
Transition function T_B={(0, 0): 1, (0, 1): 2, (0, 2): 3, (1, 0): 1, (1, 1): 1, (1, 2): 1, (2, 0): 2, (2, 1): 2, (2, 2): 2, (3, 0): 3, (3, 1): 3, (3, 2): 3}


In [32]:
#C

states = set(range(4))
actions = set(range(2))

print(f'{states=} | {actions=}')

T_С = {
    (0, 0): 1,  
    (0, 1): 2,  

    (1, 0): 1,  
    (1, 1): 3,  

    (2, 0): 3,  
    (2, 1): 2,  

    (3, 0): 3,  
    (3, 1): 3,  
}
print(f'Transition function {T_С=}')

C_mdp = states, actions, T_С


states={0, 1, 2, 3} | actions={0, 1}
Transition function T_С={(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 3, (2, 1): 2, (3, 0): 3, (3, 1): 3}


Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [33]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))

В качестве дополнения, запишите стратегию агента

In [34]:
class Agent:
    def __init__(self, actions, policy=None):
        self.rng = np.random.default_rng()
        self.actions = np.array(sorted(list(actions)))
        self.policy = policy or {}

    def act(self, state):
        if state in self.policy:
            chosen = self.policy[state]
            if isinstance(chosen, (list, np.ndarray)):
                return self.rng.choice(self.actions, p=chosen)
            else:
                return chosen
        return int(self.rng.choice(self.actions))


policy_A = {
    0: 0,  
    1: 0,  
    2: 0,  
}

policy_B = {
    0: 0,  
    1: 0,  
    2: 0,  
    3: 0,  
}

policy_C = {
    0: 0,  
    1: 1,  
    2: 0,  
    3: 1,  
}

agent = Agent(actions={0,1}, policy=policy_C)
print([agent.act(s) for s in [0,1,2,3]])


[0, 1, 0, 1]


## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [35]:

R_A = {
    (0, 0): -0.1,
    (1, 0):  1.0,
    (2, 0):  0.0
}
print("R_A =", R_A)

A_mdp = *A_mdp, R_A
print("A_mdp =", A_mdp)

R_B = {
    (0, 0):  0.5,  
    (0, 1):  1.0,  
    (0, 2): -0.2,  

    (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0,  
    (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0,
    (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0
}
print("R_B =", R_B)

B_mdp = *B_mdp, R_B
print("B_mdp =", B_mdp)



R_C = {
    (0, 0): -0.1,   
    (0, 1): -0.1,   
    (1, 0): -0.05,  
    (1, 1):  0.8,   
    (2, 0):  0.6,   
    (2, 1): -0.05,  
    (3, 0):  0.0,   
    (3, 1):  0.0
}
print("R_C =", R_C)

C_mdp = *C_mdp, R_C
print("C_mdp =", C_mdp)


R_A = {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}
A_mdp = ({0, 1, 2}, {0}, {(0, 0): 1, (1, 0): 2, (2, 0): 2}, {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0})
R_B = {(0, 0): 0.5, (0, 1): 1.0, (0, 2): -0.2, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0}
B_mdp = ({0, 1, 2, 3}, {0, 1, 2}, {(0, 0): 1, (0, 1): 2, (0, 2): 3, (1, 0): 1, (1, 1): 1, (1, 2): 1, (2, 0): 2, (2, 1): 2, (2, 2): 2, (3, 0): 3, (3, 1): 3, (3, 2): 3}, {(0, 0): 0.5, (0, 1): 1.0, (0, 2): -0.2, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0})
R_C = {(0, 0): -0.1, (0, 1): -0.1, (1, 0): -0.05, (1, 1): 0.8, (2, 0): 0.6, (2, 1): -0.05, (3, 0): 0.0, (3, 1): 0.0}
C_mdp = ({0, 1, 2, 3}, {0, 1}, {(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 3, (2, 1): 2, (3, 0): 3, (3, 1): 3}, {(0, 0): -0.1, (0, 1): -0.1, (1, 0): -0.05, (1, 1): 0.8, (2, 0): 0.6, (2, 1): -0.05, (3, 0): 0.0, (3, 1): 0.0})


## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [36]:
def run_episode(mdp):
    states, actions, T, R, term = mdp
    agent = Agent(actions)

    s = 0
    tau = []
    for _ in range(5):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

        if s in term:
            break

    return tau


Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

In [37]:
term_A = [2]
term_B = [1,2,3]
term_C = [3]

A_mdp = *A_mdp, term_A
B_mdp = *B_mdp, term_B
C_mdp = *C_mdp, term_C

# print(run_episode(A_mdp))
# print(run_episode(A_mdp))
# print(run_episode(A_mdp))
# print(run_episode(B_mdp))
# print(run_episode(B_mdp))
# print(run_episode(B_mdp))
# print(run_episode(C_mdp))
# print(run_episode(C_mdp))
# print(run_episode(C_mdp))


### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [43]:
def calculate_return(trajectory, gamma=1.0, start_idx=0):
    G = 0.0
    for i in range(start_idx, len(trajectory)):
        s, a, r = trajectory[i]
        G += (gamma ** (i - start_idx)) * r
    return G



def calculate_average_return(mdp, num_episodes=1000, gamma=1.0):
    total_returns = []
    
    for _ in range(num_episodes):
        tau = run_episode(mdp)
        if len(tau) > 0:
            # Отдача для начального состояния (s_0)
            G = calculate_return(tau, gamma, 0)
            total_returns.append(G)
    
    return np.mean(total_returns) if total_returns else 0.0


tau_A = run_episode(A_mdp)
print("Траектория для MDP A:", tau_A)
    
avg_return_A = calculate_average_return(A_mdp, num_episodes=1000, gamma=1.0)
print(f"Средняя отдача для MDP A (γ=1.0): {avg_return_A:.4f}")

tau_B = run_episode(B_mdp)
print("Траектория для MDP B:", tau_B)
avg_return_B = calculate_average_return(B_mdp, num_episodes=1000, gamma=1.0)
print(f"Средняя отдача для MDP B: {avg_return_B:.4f}")

tau_C = run_episode(C_mdp)
print("Траектория для MDP C:", tau_C)
avg_return_C = calculate_average_return(C_mdp, num_episodes=1000, gamma=1.0)
print(f"Средняя отдача для MDP C: {avg_return_C:.4f}")


Траектория для MDP A: [(0, 0, -0.1), (1, 0, 1.0)]
Средняя отдача для MDP A (γ=1.0): 0.9000
Траектория для MDP B: [(0, 2, -0.2)]
Средняя отдача для MDP B: 0.4203
Траектория для MDP C: [(0, 1, -0.1), (2, 1, -0.05), (2, 1, -0.05), (2, 0, 0.6)]
Средняя отдача для MDP C: 0.5092
